<h3>Take the file and separate all the reviews into opinion units</h3>

In [ ]:
from opinionTokenizer import tokenize_into_opinion_units

In [ ]:
import unicodecsv as csv
filename = #the name of your csv here
f = open(filename)
#divide the data into opinion units:
content = [tokenize_into_opinion_units(row[1]) for row in csv.reader(f)]
f.seek(0)
content.extend([tokenize_into_opinion_units(row[4]) for row in csv.reader(f)])

In [ ]:
opinion_units = []
for row in content:
    for elem in row:
        opinion_units.append(elem)

<h3>Get the sentiment for each opinion unit</h3>

In [ ]:
from monkeylearn import MonkeyLearn

In [ ]:
api_key = #your api key here
ml = MonkeyLearn(api_key)
module_id = 'cl_rZ2P7hbs'
res_sentiment = ml.classifiers.classify(module_id, opinion_units, sandbox=False)

Check out the results

In [ ]:
res_sentiment.result

<h3>Get the topic for each opinion unit</h3>

In [ ]:
module_id = 'cl_TKb7XmdG'
res_topic = ml.classifiers.classify(module_id, opinion_units, sandbox=False)

In [ ]:
res_topic.result

<h3>Process the results</h3>

Define a dict for storing the results.
For each entry in the dict, the first element is the number of good reviews, and the second is the number of bad ones.

In [ ]:
results = {
        "Cleanliness":[0,0],
        "Comfort & Facilities":[0,0],
        "Food":[0,0],
        "Internet":[0,0],
        "Location":[0,0],
        "Staff":[0,0],
        "Value for money":[0,0]
          }

Then, combine the classified data into the dict:

In [ ]:
for i in range(len(opinion_units)):
    for topic_dict in res_topic.result[i]:
        if res_sentiment.result[i][0]['label'] == 'Good':
            results[topic_dict[0]['label']][0] += 1
        else:
            results[topic_dict[0]['label']][1] += 1
    

Print the final results:

In [ ]:
results

Plot the final results using plotly:

In [ ]:
import plotly
from plotly.graph_objs import Bar, Layout
trace1 = Bar(
    x=['Cleanliness', 'Comfort & Facilities', 'Food', 'Internet', 'Location', 'Staff', 'Value for money'],
    y=[results['Cleanliness'][0], results['Comfort & Facilities'][0],results['Food'][0],results['Internet'][0],results['Location'][0],results['Staff'][0],results['Value for money'][0]],
    name='Positive',
    marker=dict(
        color = 'rgb(64,219,59)'
    )
)
trace2 = Bar(
    x=['Cleanliness', 'Comfort & Facilities', 'Food', 'Internet', 'Location', 'Staff', 'Value for money'],
    y=[results['Cleanliness'][1], results['Comfort & Facilities'][1],results['Food'][1],results['Internet'][1],results['Location'][1],results['Staff'][1],results['Value for money'][1]],
    name='Negative',
    marker=dict(
        color = 'rgb(235,54,72)'
    )
)

plotly.offline.plot({
"data": [
            trace1, trace2
],
"layout": Layout(
    title="Reviews by topic",
    barmode="group"
)
})